This notebook produces an interactive plot of the LSST MAF skymap results for our bayes factor metric alongside the median bayes factor for each region of sky, to demonstrate the connection between the two.  

In [ ]:
%matplotlib notebook 
#ABOVE: enables matplotlib interactivity inside the notebook
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.widgets import Slider

ATTENTION: Change the band string to switch bands (u or g)

In [ ]:
band = 'u'
if band != 'u' and band != 'g':
    print 'ERROR: Band must be either u or g'

This cell loads and splits the datafiles into arrays for easy plotting. 

In [ ]:
galband = np.loadtxt('galaxband-'+band+'-medianbayes.dat', usecols=range(2,1)) #galactic band
galpole = np.loadtxt('galaxpole-'+band+'-medianbayes.dat', usecols=range(2,1)) #galactic pole
mainsurvey = np.loadtxt('main-'+band+'-medianbayes.dat', usecols=range(2,1)) #main survey
if band == 'g': #the ecliptic spur is not present in the u band
	eclip = np.loadtxt('ecliptic-g-medianbayes.dat', usecols=range(2,1)) #ecliptic spur
	ecliptic, eclzshift = np.split(eclip, 2, axis = 1)
deepdr = np.loadtxt('deepdrill-'+band+'-medianbayes.dat', usecols=range(2,1)) #deep drilling fields
galaxband, bandzshift = np.split(galband, 2, axis=1)
galaxpole, polezshift = np.split(galpole, 2, axis=1)
main, mainzshift = np.split(mainsurvey, 2, axis=1)
deepdrill, deepzshift = np.split(deepdr, 2, axis=1)

This cell defines the update function for the slider, which plots the appropriate skymaps based on the slider's value.

In [ ]:
oldval = 0.1 #this is the initial value of the slider
def update(val):
    global oldval
    if szshift.val >= oldval + 0.05 or szshift.val <= oldval - 0.05: #check that we've moved far from the currently displayed plot
        #clear old plots
        ax.cla()
        imgplot.cla()
        imgplot2.cla()
        #reset label
        ax.set_ylabel('Log(Median Bayes Factor)')
        ax.plot(deepzshift, np.log(deepdrill), label="deep drill fields")
        ax.plot(bandzshift, np.log(galaxband), label="galactic band")
        ax.plot(polezshift, np.log(galaxpole), label="galactic pole")
        ax.plot(mainzshift, np.log(main), label = "main survey")
        if band == 'g': #the ecliptic spur is not present in the u band
            ax.plot(eclzshift, np.log(ecliptic), label="ecliptic spur")
            ax.plot((round(szshift.val, 1)), np.log(ecliptic[int((round(szshift.val, 1)*10)-1)]), 'ko')
        ax.plot((round(szshift.val, 1)), np.log(deepdrill[int((round(szshift.val, 1)*10)-1)]), 'ko')
        ax.plot((round(szshift.val, 1)), np.log(main[int((round(szshift.val, 1)*10)-1)]), 'ko')
        ax.plot((round(szshift.val, 1)), np.log(galaxpole[int((round(szshift.val, 1)*10)-1)]), 'ko')
        ax.plot((round(szshift.val, 1)), np.log(galaxband[int((round(szshift.val, 1)*10)-1)]), 'ko')
        ax.legend(prop={'size':6})
        ax.xaxis.set_visible(False)
        #reset filepaths
        imgpath = 'MAFthumbs/z' + str(round(szshift.val, 1)) + '/' + band + 'band/'
        imgname = 'thumb.opsim_astrometry_' + band + '_HEAL_kraken_1042_SkyMap.png'
        imgname2 = 'thumb.opsim_astrometry_' + band + '_and_night_lt_366_HEAL_kraken_1042_SkyMap.png'
        img = mpimg.imread(imgpath + imgname)
        img2 = mpimg.imread(imgpath + imgname2)
        #plot skymap thumbnails for nearest applicable value
        imgplot.imshow(img)
        imgplot2.imshow(img2)
        oldval = szshift.val #store a new value to check against
    fig.canvas.draw_idle()

This cell sets up the initial condition of the plot and starts the interactive slider widget.  Note that there are only skymaps for each 0.1 interval of redshift, so not every value on the slider has a skymap associated.  This results in apparent inaccuracies, especially in the areas halfway between two skymaps.  

In [ ]:
fig = plt.figure(figsize=(10,10))
fig.suptitle(band + '-band', fontsize=16)
ax = fig.add_subplot()
ax = plt.subplot2grid((8,6), (0,0), colspan = 6, rowspan = 2) #plots to the upperleft square in a 4x6 grid, 6 columns long, 2 rows deep
ax.set_ylabel('Log(Median Bayes Factor)')
ax.plot(deepzshift, np.log(deepdrill), label="deep drill fields") #plot of median bayes factor in deep drilling fields
ax.plot(bandzshift, np.log(galaxband), label="galactic band") #ditto for the galactic band
ax.plot(polezshift, np.log(galaxpole), label="galactic pole") #ditto for the galactic pole
ax.plot(mainzshift, np.log(main), label = "main survey") #ditto for the main survey area
if band == 'g': #the ecliptic spur is not present in the u band
	ax.plot(eclzshift, np.log(ecliptic), label="ecliptic spur") #ditto for the ecliptic spur
ax.plot(oldval, np.log(deepdrill[int((oldval*10)-1)]), 'ko') #plots the initial point
ax.legend(prop={'size':6})
ax.xaxis.set_visible(False)
#setup for image handling; imgplot is 10 year, imgplot2 is 1 year
imgplot = plt.subplot2grid((8,6), (2, 3), colspan = 3, rowspan = 5) #4x6 grid, 3rd row, 4th column, 3 wide, 2 deep
imgplot.xaxis.set_visible(False)
imgplot.yaxis.set_visible(False)
imgplot2 = plt.subplot2grid((8,6), (2, 0), colspan = 3, rowspan = 5) #4x6 grid, 3rd row, 1st column, 3 wide, 2 deep
imgplot2.xaxis.set_visible(False)
imgplot2.yaxis.set_visible(False)
f0 = oldval #initial value of the slider/redshift
#filepaths for skymap thumbnails
imgpath = 'MAFthumbs/z' + str(f0) + '/' + band + 'band/'
imgname = 'thumb.opsim_astrometry_' + band + '_HEAL_kraken_1042_SkyMap.png'
imgname2 = 'thumb.opsim_astrometry_' + band + '_and_night_lt_366_HEAL_kraken_1042_SkyMap.png'
img = mpimg.imread(imgpath + imgname)
img2 = mpimg.imread(imgpath + imgname2)

imgplot.imshow(img)
imgplot2.imshow(img2)
#setup for slider
axcolor = 'lightgoldenrodyellow' 
axfreq = plt.axes([0.16, 0.68, 0.705, 0.02], axis_bgcolor=axcolor) #if this raises an error, try changing "axis_bgcolor" to "facecolor"
sl_xticks = np.arange(0.1,3.9, 0.2)
szshift = Slider(axfreq, '', 0.1, 3.9, valinit=f0)
axfreq.xaxis.set_visible(True)
axfreq.set_xticks(sl_xticks)
szshift.on_changed(update)
plt.show()

The slider, despite its name, isn't very good at sliding.  It's much better to just click on the slider where you want to see, rather than sliding to it.  